# Autocomplete  

A model for predicting the next word in a sentence

### Overview

We aim to build an application that predicts the next word of a user's query - Naturally, this will involve dealing with probabilities. To build our predictor, we'll be implementing a Markov Model as our stochastic tool of choice.

Before getting into the details, let's take a look at the big picture. Our application will be taking in the user's input text, and will need some training set of data in order to sample probabilities of word sequences.  

High-Level view of application:
- Input: User's query, training set
- Output: Next word  

This leads us to make a function like *predict* below

In [ ]:
def predict(query, data):
    # Return the most likely next word
    return

However, we'll be using the data to train the model that we're creating. And the model itself will be the thing that is used to determine the next word. Thus, we'll be implementing the more verbose version of *predict* below

In [ ]:
def predict(query, model):
    # Return the most likely next word
    return

### Markov Model

We now implement a version of a markov model that fits our problem statement. Each state in our markov chain will represent a unique word, and the probability that we go from state *A* to state *B* will represent the probability that the next word is *B* given the current word *A*.

In [ ]:
class Markov():
    
    def __init__(self, data=None):
        self.word_history = {}
        self.next_word_map = {}
        self.data = data
    
    def train_word(self, word, next_word):
        word = word.lower()
        next_word = next_word.lower()
        word_dict = self.word_history.setdefault(word, {next_word : 0})
        word_dict.setdefault(next_word, 0)
        word_dict[next_word] += 1
        self.update()
    
    def train_set(self, words, domain=0):
        # implement for data input in form of text file
        # possible forms of words... .txt, string (sentence), etc
        if domain == 0:
            domain = len(words)-1
        for i in range(domain):
            self.train_word(words[i], words[i+1])
    
    def update(self):
        for word in self.word_history:
            most_probable = 0
            best_next = "..."
            for key, value in self.word_history[word].items():
                if value > most_probable:
                    best_next = key
                    most_probable = value
            self.next_word_map[word] = best_next
    
    def retrieve_next(self, word):
        return self.next_word_map.setdefault(word, "...")

    def probability(self, word, next_word):
        word = word.lower()
        next_word = next_word.lower()
        total = sum([val for val in self.word_history[word].values()])
        return 1.0*self.word_history[word][next_word]/total


We create our Markov class to represent the Markov chain object that we need to create

Class Methods  

~~~~
__init__
    Initializes a new instance of a Markov chain

train_word
    trains the model using a pair of words

train_set
    trains the model using a list of words (multiple pairs of words)

update
    updates the stored probabilities after a new word is introduced

retrieve_next
    retrieves the next predicted word based on the previous word

probability
    returns the probability that next_word will follow word

~~~~

### Application

Finally, we incorporate this into our original function by updating *predict*. We simplify our task by focusing on the last word of the user's query.

I.e., for a query "Hi my name is", we would look for the most likely word to follow "is".

This simplification is made to accomodate for what our currently model is capable of, but we can see that a more powerful next-word-predictor would require a more sophisticated model.

In [ ]:
def predict(query, model):
    words = query.split(" ")
    return model.retrieve_next(words[-1])

### Tests

Now that we've built our application, let's learn more about how it works by using it on some actual input.

In [ ]:
test_model = Markov()
test_model.train_word("hello", "everyone")
print(test_model.word_history)

In [ ]:
print(test_model.next_word_map)

In [ ]:
test_model.train_word("hello", "world")
print(test_model.word_history)

In [ ]:
test_model.train_word("hello", "world")
print(test_model.word_history)

In [ ]:
print(test_model.next_word_map)

In [ ]:
test_model.train_word("goodbye", "everyone")
print(test_model.word_history)

In [ ]:
print(test_model.next_word_map)

We've trained our model with a modest training set, but even this tiny sample size is enough for our model to start making some predictions.  

After running the code above, we get the following counts:  

hello -> world : 2  
hello -> everyone : 1  
goodbye -> everyone : 1  

This is enough to tell us that if our last word was "hello", there is a 66% chance that the next word will be "world", and a 33% chance that the next word will be "everyone".  
Similarly, a query of "goodbye" would result in "everyone" 100% of the time.  

Since our application chooses the most likely next word, we can expect that given an input of "hello", it would ouput "world". Likewise, an input of "goodbye" would result in "everyone".


In [ ]:
print("Input: hello")
print("Output: " + predict("hello", test_model))
print("Probability: " + str(test_model.probability("hello", "world")))

print("\n")

print("Input: goodbye")
print("Output: " + predict("goodbye", test_model))
print("Probability: " + str(test_model.probability("goodbye", "everyone")))

Now that we've tested the application with a smaller data set, let's use a more complete sequence of words to train our model. Looking for a corpus of text brings to mind NLTK, which we'll use to import the full text to the screenplay of Pirate of the Carribean.

In [ ]:
import nltk
from nltk.corpus import webtext

pirates = webtext.words('pirates.txt') # creating a list where each element is a word (in order from the screenplay)

new_model = Markov()
new_model.train_set(pirates, 3000)
print("Done")

We've taken the text of "Pirates of the Caribbean" and used the first 3000 words to train our model. Let's see what our model would predict the characters of this movie to say.

In [ ]:
# Input = Jack, Output = ???
print(predict("jack", new_model))
print("Probability: " + str(new_model.probability("jack", predict("jack", new_model))))

Now, let's see what would be predicted if we were to continue predicting words to form a sentence... In other words, given the ability to generate the next word, we should be able to contruct a phrase or sentence as well.

In [ ]:
query = "pirate"
print("Next word: " + predict(query, new_model))
print("Probability: " + str(new_model.probability(query, predict(query, new_model))))

query = query + " " + predict(query, new_model)
print("Sentence: " + query)


In [ ]:
print("Next word: " + predict(query, new_model))

query = query + " " + predict(query, new_model)
print("Sentence: " + query)

In [ ]:
print("Next word: " + predict(query, new_model))

query = query + " " + predict(query, new_model)
print("Sentence: " + query)

Next Steps: evaluate whether a sentence makes sense or not

for each word in a sentence, use probability that the next word follows it

given several sentence, which one makes the most sense? which one makes the least sense?

further applications of NLTK?


